# Analysis of mouse single cell hematopoietic populations - Hierarchical clustering of index sorted CMP
__Author__: Elisabeth F. Heuston

## Purpose

Single cell transcriptional and clustering analysis of LSK, CMP, MEP, and GMP data presented in Heuston et al., 2021  

raw data are available at

## Updates

### Update 2021.07.19:  
Previous version (IndexSorted_CMP-archive2.ipynb) was last version to use original 2018/2019 Ct thresholds. New analysis described here:  
* Using Fluidigm Real-Time PCR Analysis v4.7.1 by Fluidigm Corporation  
* Set Quality Threshold to 0.1, and go through everything that doesn't pass (manually I am very generous)  
* Set Ct threshold Method to Auto (Detectors) -- This means that the dRN (Ct theshold) is calculated *per Probe* rather than globally. In my mind this means we can still use probes that may have overall weaker amplification than others  
* Rule of thumb is that we pass amplfication that crosses Ct threshold at most twice  
* Set baseline correction at Linear (Derivative). This is unchanged from previous analyses

### Update:  
Previous version (IndexSorted_CMP-archive.ipynb) loaded data, performed normalization, then filtered cells to include only those that were in both Biomark and Flow. This means that empty wells or wells with 2 cells are included in normalization and might offset scales.  
Updated version will:  
* Load RNA and MFI data  
* Filter cells for QC
* Filter cells with both RNA and MFI data
* Perform RNA and MFI normalizations

## Workbook setup

## Load modules

In [ ]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter, ExcelFile
import matplotlib.pyplot as plt
import sklearn.preprocessing as pp
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.colors import DivergingNorm
from sklearn.manifold import TSNE
import seaborn as sns
from pylab import savefig
from scipy.cluster.hierarchy import dendrogram, linkage
import os
import re
from icecream import ic

## User-defined functions


In [ ]:
def generate_probe_statistics(df, sheetname, writer):
    stats_table=pd.DataFrame()
    stats_table['Failed Probe'] = df.sum(axis=1)==0
    stats_table['sum'] = df.sum(axis=1).round(2)
    stats_table['min Ct'] = df.min(axis=1).round(2)
    stats_table['max Ct'] = df.max(axis=1).round(2)
    stats_table['mean Ct'] = biomark_ctIndexed.mean(axis=1).round(2)
    stats_table['median Ct'] = df.median(axis=1).round(2)
    stats_table['Ct std'] = df.std(axis=1).round(2)
    stats_table.to_excel(writer, sheetname)

### apply_ct_limits

In [ ]:
def apply_ct_limits(x, lowerlimit, upperlimit):
    if x < lowerlimit or x > upperlimit:
        return 0
    else:
        return x

### dCt

In [ ]:
def dCt (plate, ref_list = ["Actb", "B2m", "Cd117"]):

    failed = []
    [failed.append(x) for x in ref_list if not rna_df.columns.str.contains(x).any()]
    
    # Test if table in correct format
    if bool(failed):
        print('None of', failed, 'found in columns')        
        
    refCt = plate[plate[ref_list] > 0].mean(axis = 1)
    plate = plate.subtract(refCt, axis = 0)
    return plate

### cell_filter

In [ ]:
def cell_filter(plate, lowerlimit = 8, upperlimit = 40, min_passed_tests = 30, min_passed_references = 2, avg_ct_thresh = None, transpose_plate = True):
    
    # Read plate
    plate.columns = ['Well ID', 'Probe', 'Ct']
    plate['Ct'] = plate['Ct'].replace(999.0, np.nan) # replace fails with NA
    
    # Filter cells
    plate['Ct'] = plate['Ct'].apply(lambda x: apply_ct_limits(x, lowerlimit, upperlimit)) # apply Ct threshold limits
    plate = plate.set_index(['Well ID', 'Probe']) # clean index
    plate = plate.unstack(level = 'Well ID') # clean index
    plate.columns = plate.columns.droplevel() # clean index
    plate = plate.transpose() # Sets cells to index
    
    if not min_passed_tests == None: # Drop cells where fewer than X probes passed
        plate = plate[plate[plate > 0].count(axis = 1) >= min_passed_tests] 
    
    if not min_passed_references == None: # Drop cells where fewer than X reference probes passed
        plate = plate[plate[plate[["Actb", "B2m", "Cd117"]] > 0].count(axis = 1) >= min_passed_references] 
        
    if not avg_ct_thresh == None: # Drop cells where mean Ct > theshold
        plate = plate[plate[plate > 0].mean(axis = 1) >= avg_ct_thresh] 
        
        
    plate = plate.replace(0, np.nan) # replace all 0 values with NA

    if transpose_plate == False:
        plate = plate.transpose()
    return plate

### probe_filter

In [ ]:
def probe_filter(plate, pct_nan_allowed = .2):
    plate = plate.dropna(thresh=int(len(plate)*pct_nan_allowed), axis = 1)
    return plate

### dCt_xprsn

In [ ]:
def dCt_xprsn(plate):
    plate = 2**(plate*-1)
    plate = plate.replace(np.nan, 0)
    return plate

### zscore_norm

In [ ]:
def zscore_norm (plate):
    
    #Calculate z_score
    row_mean = plate.mean(axis = 1, skipna = True)
    row_std = plate.std(axis = 1, skipna = True)
    plate = plate.sub(row_mean, axis = 0)
    plate = plate.div(row_std, axis = 0)

    return plate

### flow_norm

In [ ]:
def flow_norm(plate):
    
    # Calculate z_score (x - mean/stdev)
    column_mean = plate.mean(axis = 0, skipna = True)
    column_std = plate.std(axis = 0, skipna = True)
    plate = plate.sub(column_mean, axis = 1)
    plate = plate.div(column_std, axis = 1)
    
    # Replace 0 with NA
    plate = plate.replace(0, np.nan)
    return plate  

In [ ]:
def binning_flow(indexStats_column):
    if indexStats_column.mean() !=0:
        flow_bins = [-np.inf, 0, indexStats_column.mean(), indexStats_column.mean()+2*indexStats_column.std(), np.inf]
        flow_labels = [0,1,2,3]
    elif indexStats_column.mean() == 0:
        if indexStats_column.std() ==0:
            flow_bins = [-np.inf, np.inf]
            flow_labels = [0]
        elif indexStats_column.std() !=0:
            flow_bins = [-np.inf, indexStats_column.mean(), indexStats_column.mean()+2*indexStats_column.std(), np.inf]
            flow_labels = [0, 1, 2]
    else:
        return
    binned_flow = pd.cut(indexStats_column, bins = flow_bins, labels = flow_labels)
    return pd.Series(binned_flow)

### assign_plate_id

In [ ]:
def assign_plate_id(col, sortdate, sort_dict):
    pid = ''
    if sortdate in ['010319', '121218', '020119']:
        if any(col.str.contains('01')):
            pid = sort_dict[sortdate][0]
        elif any(col.str.contains('07')):
            pid = sort_dict[sortdate][1]
        else:
            pid = 'unknown'
    elif sortdate == '090618':
        if any(col.str.contains('01')):
            pid = sort_dict[sortdate][1]
        elif any(col.str.contains('07')):
            pid = sort_dict[sortdate][0]
        else:
            pid = 'unknown'
    else:
        pid = "notInSortList"
    return pid

In [ ]:
heatmap_colors = sns.cubehelix_palette(n_colors = 10, start = 0.5, rot = -0.8, gamma = 0.6, hue = 1.00, light = .9, dark = 0, reverse = True)

# Read RNA and MFI data

## RNA

### Plate translations  

Plate|PlateID|SortDate|RunDate|Pop1|Pop2| 
-----|-------|--------|-------|----|----|
Plate1|LM_Plate1|01.03.19|02.15.19|CMP|MEP|  
Plate2|LM_Plate2|12.12.18|02.26.19|CMP|LSK|  
Plate3|LM_Plate3|01.03.19|03.06.19|MEP|CMP|  
Plate4|LM_Plate4|12.12.18|03.13.19|LSK|CMP|  
Plate5|LM_Plate5|01.10.19|03.18.19|LSK|LSK|  
Plate6|LM_Plate6|02.01.19|03.20.19|CMP|MEP|  
Plate7|LM_Plate7|02.01.19|08.06.19|MEP|CMP|  
Plate8|LM_Plate8|02.06.19|08.07.19|LSK|MEP|  
Plate9|LM_Plate9|01.23.19|08.09.19|LSK|MEP|  
Plate10|LM_Plate10|02.06.19|08.14.19|MEP|LSK|  
Plate11|LM_Plate11|01.23.19|08.19.19|MEP|LSK|  
Plate12|BP1|09.06.18|10.23.18|LSK|CMP|  
Plate13|BP2|09.06.18|10.23.18|CMP|LSK|  

Open every sheet in excel file, keep only rows where name contains CMP, then do basic QC

### Read RNA

In [ ]:
excel_biomark = pd.ExcelFile("/Users/heustonef/Desktop/Github/MouseSingleCellPaper/BioMark_CT.xlsx")

rna_list = []
for sheet in excel_biomark.sheet_names:
    plate_data = pd.read_excel(excel_biomark, sheet_name=sheet, skiprows=11, usecols="B, E, G")
    plate_data = plate_data[plate_data["Name"].str.contains("CMP")]
    if not plate_data.empty:
        plate_data["Name"] = plate_data["Name"] + '_' + ''.join(re.search('(\w)[a-zA-Z]*(\d+)', sheet).groups([0, 2])).lower()
        plate_data = cell_filter(plate_data, lowerlimit=8, upperlimit=40, min_passed_tests=2, avg_ct_thresh=None, transpose_plate=True)
        rna_list.append(plate_data)
        
rna = pd.concat(rna_list, axis = 0, ignore_index = False)
rna = probe_filter(rna, pct_nan_allowed=0.2)

print("Number of cells:", rna.shape[0])
print("Number of probes:", rna.shape[1])
rna

### RNA index

In [ ]:
rna_idx = set(rna.index)

## MFI

In [ ]:
sort_dict = {'010319':['p1', 'p3'], 
            '121218': ['p2', 'p4'],
            '020119': ['p6', 'p7'],
            '090618': ['p12', 'p13'],
           }

In [ ]:
fluor_dict = {'FITC-A': 'Cd41', 
              'PerCP-A': 'Cd135', 
              'PE-A': 'Cd123', 
              'PE-Cy7-A': 'Cd48', 
              'APC-A': 'Cd34', 
              'APC-Alexa 700-A': 'Cd16/32', 
              'BV510-A': 'Viability', 
              'BV650-A': 'Cd150', 
              'BV786-A': 'Cd36', 
              'PI-A': 'Cd117',
              'PerCP-Cy5-5-A': 'Cd135',
              'BV421-A': 'Cd9',
              'BV750-A': 'Cd36', 
              'PE-CF594-A': 'Cd117', 
             }

### Ab panel for CMP

Antibody|Fluor|Equivalencies|
--------|-----|-------------|
Cd36|SB780|BV786|
Flk2/Flt3/Cd135|PerCp-eFluor710|PerCpCy5.5|
Cd48|PE-Cy7|N/A|
Cd9|eFluor450|BV421/SB436|
Cd34|eFluor660|APC|
Cd123|PE|N/A|
cKit/Cd117|PE-eFluor610|PI|
Cd16/32|AF700|APC-Alexa700|
Cd41|Fitc|N/A|
Cd150|SB645|BV650|
Viability|eFluor506|AmCyan|

### Read MFI

In [ ]:
plates_01_03 = "/Users/heustonef/Desktop/CMPSubpops/FlowData/010319 EH CMP extended violet panel/"
plates_02_04 = "/Users/heustonef/Desktop/CMPSubpops/FlowData/121218 EH CMP extended violet panel/"
plates_12_13 = "/Users/heustonef/Desktop/CMPSubpops/FlowData/090618 CMP Ext Vio FWDS 070/"
plates_06_07 = "/Users/heustonef/Desktop/CMPSubpops/FlowData/020119 EH CMP extended violet panel/"

cmp_plates = [plates_01_03, plates_02_04, plates_06_07, plates_12_13]

In [ ]:
list_of_files = []
for plate in cmp_plates:
    for (dirpath, dirnames, filenames) in os.walk(plate):
        for filename in filenames:
            if filename.endswith('Well.csv'): 
                list_of_files.append(os.sep.join([dirpath, filename]))

Note that unlike RNA, MFI data just needs to be read in. Filtering based on detection of non-CMP markers (i.e., cKit, Cd16/32, Cd34, viability) was done in Flow Jo

In [ ]:
mfi = pd.DataFrame()
mfi_list = []

leading_zeros = re.compile('(?<=[a-z])0(?=[0-9])')

for file in list_of_files:
    df = pd.read_csv(file, usecols=[0, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
    df = df.rename(columns = fluor_dict)
    sortdate = re.search('(^\d+).*(\d{3}_\d{3})', os.path.basename(file)).group(1)
    plateid = re.search('(^\d+).*(\d{3}_\d{3})', os.path.basename(file)).group(2)
    pid = assign_plate_id(df.Well, sortdate, sort_dict)
    df['sortdate'] = sortdate
    df['plateid'] = plateid
    df['pid'] = pid
    df['Well'] = df['Well'].str.lower()
    df['Well'] = df['Well'].map((lambda x: re.sub(leading_zeros, '', x)))
    mfi_list.append(df)
mfi = pd.concat(mfi_list, axis = 0, ignore_index=True)

mfi = mfi.sort_values(['sortdate', 'Well', 'plateid'])
mfi = mfi.drop_duplicates(subset = ['Well', 'sortdate'], keep = 'first')
mfi.index = 'CMP_' + (mfi['Well'].str.cat(mfi['pid'], sep = '_')).astype('str')
mfi = mfi.drop(['Cd16/32', 'Viability'], axis = 1)
for fluor in mfi.columns:
    if re.search(r'\d+$', fluor):
        mfi = mfi.rename(columns = {fluor: (fluor + 'pos')})

mfi.shape

#### Find and assign missing pids

In [ ]:
unk_list = np.where(mfi.pid == 'unknown')[0].tolist()
unk_list

In [ ]:
for unk in unk_list:
    print(unk)
    print(mfi.iloc[(unk-1):(unk+2), [0, 10, 11, 12]])

In [ ]:
mfi = mfi.rename(index = {'CMP_d9_unknown': 'CMP_d9_p7'})
mfi.pid[142] = 'p7'

In [ ]:
mfi

### MFI index

In [ ]:
mfi_idx = set(mfi.index)

# Filter RNA and MFI cell sets

## Intersect RNA and MFI indices

In [ ]:
shared_idx = rna_idx.intersection(mfi_idx)

In [ ]:
len(shared_idx)

## RNA df
Keep only cells in RNA and MFI

### Filter

In [ ]:
rna_df = rna.loc[rna.index.isin(shared_idx), ]

### Normalize 

In [ ]:
rna_df.head()

Check standard deviation of probes compared to what we're using now

In [ ]:
rna_df = dCt(rna_df)

In [ ]:
[print(x, '\t',  round(rna_df[x].std(), 3)) for x in rna_df.columns if rna_df[x].std() < max(rna_df['Actb'].std(), rna_df['B2m'].std(), rna_df['Cd117'].std())]

Looks like in addition to Actb, B2m, and Cd117, there are 4 additional probes (Cst3, Hp, Ifitm1, Prtn3) with low standard deviation. Might add these later.  

>**NB:**  
These are low std *after* dCt calculation. The list is much longer if you test before you do the correction.

#### dCt normalization

Using deltaCt method:  
> refGenes = Act, B2m, Cd117  
> refCt = average(Actb, B2m, Cd117) (for refGenes > 0)  
> deltaCt(gene) = Ct(gene) - refCt  

So deltaCt method normalizes individual cells to reference genes, so plate normalization is not necessary.  
Probe filtering has removed uninformative probes.  
For the moment let's ignore flow data (which probably has to be reanalyzed) and cluster just the cells

##### 2^-dCt

Delta delta Ct needs comparison against a control, but we don't have (i.e.,) treated vs untreated. Could use ref genes again, but this is probably double-normalizing.  
Let's just do expression of the dCts

In [ ]:
rna_xprsn = dCt_xprsn(rna_df)

##### Hierarchical clustering

Optional clustering methods:  
> single  
> complete  
> average  
> weighted  
> centroid  
> median  
> ward  

In [ ]:
sns.clustermap(rna_xprsn.transpose(), figsize=(25,15), method = 'average', robust = True, xticklabels = False, yticklabels = True, vmin = 0)
sns.clustermap(rna_df.replace(np.nan, 0).transpose(), figsize=(25,15), method = 'average', robust = True, xticklabels = False, yticklabels = True, vmin = 0)

Can color individual leaves to show which markers correspond with which celltypes  
https://python-graph-gallery.com/405-dendrogram-with-heatmap-and-coloured-leaves/

#### Z_score norm

A note about z_scores:  
> Z scores are: z = (x - mean)/std, so values in each row will get the mean of the row subtracted, then divided by the standard deviation of the row.  
> This ensures that each row has mean of 0 and variance of 1.  

Either 0 (rows) or 1 (columns):  
> 0 = Set the mean of each __gene__ to zero and see which cells express it the most  
> 1 = Set the mean of each __cell__ to zero and see which genes are most variable in it  

Z score across cells.  
Do you do z_score then dCt, or dCt then z_score, or just z_score?  
I think z_score is the cell norm method, so will try just z_score

In [ ]:
rna_zscore = dCt(rna_df)
rna_zscore = zscore_norm(rna_zscore)
rna_zscore = dCt_xprsn(rna_zscore).transpose()

##### Hierarchical clustering

In [ ]:
sns.clustermap(rna_zscore, figsize=(25,15), method = 'average', robust = True, xticklabels = False, yticklabels = True, vmin = 0)

#### Sklear.StandardScaler()

I guess if you assume each plate has the same distribution of cell types, then you can assume that plates represent a gaussian distribution of an expression profile.  
So we can scale each plate, which should make inter-plate comparisons more fair

In [ ]:
from sklearn import preprocessing

__Is this normalizing across each cell??__

In [ ]:
rna_df_by_pid = rna_df.groupby('pid')

In [ ]:
rna_scaled_list = []

for pid in rna_df_by_pid.groups.keys():
    
    df = rna_df_by_pid.get_group(pid).transpose()
    df = df.drop('pid', axis = 0)
    
    scaler = preprocessing.StandardScaler().fit(df)
    df_scaled = scaler.transform(df)
    df_scaled = pd.DataFrame(df_scaled, index = df.index, columns=df.columns)
#     ic(df_scaled.shape[0], df_scaled.mean(axis = 0), df_scaled.std(axis = 0))
    
    rna_scaled_list.append(df_scaled.transpose())
    
    
rna_scaled = pd.concat(rna_scaled_list)
# rna_scaled = rna_scaled.replace(np.nan, 0)

##### Hierarchical clustering

In [ ]:
sns.clustermap(rna_scaled.transpose(), figsize=(25,15), method = 'ward', robust = True, xticklabels = True, yticklabels = True, vmin = 0)

#### Sklearn.RobustScaler()

In [ ]:
rna_df_by_pid = rna_df.groupby('pid')

In [ ]:
rna_robust_list = []

for pid in rna_df_by_pid.groups.keys():
    
    df = rna_df_by_pid.get_group(pid).transpose()
    df = df.drop('pid', axis = 0)
    
    scaler = preprocessing.RobustScaler().fit(df)
    df_scaled = scaler.transform(df)
    df_scaled = pd.DataFrame(df_scaled, index = df.index, columns=df.columns)
#     ic(df_scaled.shape[0], df_scaled.mean(axis = 0), df_scaled.std(axis = 0))
    
    rna_robust_list.append(df_scaled.transpose())
    
    
rna_robust = pd.concat(rna_robust_list)

##### Hierarchical clustering

In [ ]:
sns.clustermap(rna_robust.transpose(), figsize=(25,15), method = 'ward', robust = True, xticklabels = True, yticklabels = True, vmin = 0)

RobustScaler is supposed to handle outliers better, but now I get a tv screen. And p13 is clumping like a beast...     
__Perform an outlier test before deciding on these results__

## Normalization notes  
[Fluidigm](https://www.fluidigm.com/faq/ge-63) suggests normalizing my _median_ Ct  
Should reinvestigate the biomark software itself and make absolutely sure there's no way to normalize through that...

### Quantile normalization

## MFI df

### Filter

In [ ]:
mfi_df = mfi.loc[mfi.index.isin(shared_idx), ]

### Hitogram of unmodified flow data

In [ ]:
mfi_by_pid = mfi_df.groupby('pid')
mfi_by_pid.groups.keys()

In [ ]:
for plate in mfi_by_pid.groups.keys():
    plot_group = mfi_by_pid.get_group(plate)
    plot_group = plot_group.replace(np.nan, 0)
    
    for array in plot_group.hist(bins = 40, layout = (9, 11), figsize = (50, 50)):
        for subplot in array:
            subplot.set_ylabel(plate)
#             subplot.set_xlim(left = 0)


In [ ]:
rdbu = sns.diverging_palette(250, 10, s = 100, l=50, sep = 1,  center= "dark", as_cmap=True)

### MFI above background

Problem with zscore norm is that it doesn't necessarily preserve fluourescence distribution  
Went throught each experiment and calculated minimum MFI that was still "positive" above background (i.e., Cd34 MFI 161 = Cd34- but Cd34 MFI 162 = Cd34+)  
For ease, going to just call this "background"  
Will subtract background from fluor for each experiment, then normalize between 1 and 100 <- this will preserve distribution of "positive" values

#### Define backgrounds for each experiment

In [ ]:
bkgd_dict = {'010319': {'Cd34pos': 162, 'Cd117pos': 1200, 'Cd9pos': 1100, 'Cd36pos': 1300, 'Cd41pos': 433, 'Cd48pos': 89, 'Cd123pos': 314, 'Cd150pos': 415, 'Cd135pos': 433},
             '020119': {'Cd34pos': 222, 'Cd117pos': 356, 'Cd9pos': 1100, 'Cd36pos': 204, 'Cd41pos': 430, 'Cd48pos': 156, 'Cd123pos': 305, 'Cd150pos': 419, 'Cd135pos': 27},
             '090618': {'Cd34pos': 593, 'Cd117pos': 419, 'Cd9pos': 376, 'Cd36pos': 828, 'Cd41pos': 431, 'Cd48pos': 197, 'Cd123pos': 960, 'Cd150pos': 618, 'Cd135pos': 1000},
             '121218': {'Cd34pos': 205, 'Cd117pos': 454, 'Cd9pos': 593, 'Cd36pos': 1300, 'Cd41pos': 755, 'Cd48pos': 91, 'Cd123pos': 1600, 'Cd150pos': 439, 'Cd135pos': 445}
            }

In [ ]:
mfi_by_sortdate = mfi_df.groupby('sortdate')
mfi_by_sortdate.get_group('010319').head(2)

In [ ]:
mfi_sortdate_list = []

for sortdate in mfi_by_sortdate.groups.keys():
    mfi_subbkgd = mfi_by_sortdate.get_group(sortdate)
    sortdate_math = bkgd_dict[sortdate]
    for fluor in sortdate_math.keys():
        
        mfi_subbkgd[fluor] = mfi_subbkgd[fluor] - sortdate_math[fluor] # sets min positive value to 0
#         ic(fluor, sortdate, 
#            mfi_subbkgd[fluor].min(), 
#            mfi_subbkgd[fluor].max()
#           )
        mfi_subbkgd[fluor] = mfi_subbkgd[fluor].clip(lower = 0)
        mfi_subbkgd[fluor] = mfi_subbkgd[fluor]/mfi_subbkgd[fluor].max()
    
    mfi_sortdate_list.append(mfi_subbkgd)
mfi_sortdate = pd.concat(mfi_sortdate_list)

In [ ]:
mfi_sortdate.head(2)

#### Histogram of corrected data

In [ ]:
mfi_sortdate_by_sortdate = mfi_sortdate.groupby('sortdate')
for plate in mfi_sortdate_by_sortdate.groups.keys():
    plate_df = mfi_sortdate_by_sortdate.get_group(plate)
    plate_df = plate_df.drop(['Well', 'sortdate', 'plateid', 'pid'], axis = 1)
#     ic(plate, 
#        plate_df.min(), 
#        plate_df.max()
#       )
    plate_df = pd.melt(plate_df).rename(columns={'variable': 'fluor', 'value': 'norm_mfi'})
    
    # create figures
    g = sns.FacetGrid(plate_df, col = 'fluor')
    g.fig.set_size_inches(30, 7)
    g.map(sns.histplot, 'norm_mfi', bins = 40, stat = 'probability', kde = True, line_kws = {'linewidth':4})
    g.set_axis_labels(y_var=plate)
    plt.xlim(left = 0)
    plt.ylim(top = 0.3)
    
#     #save figures
#     save_title = ''.join(('bkgd_norm_mfi-biomarkCells-', plate, '.png'))
#     plt.savefig(save_title)

In [ ]:
rdbu = sns.diverging_palette(250, 10, s = 100, l=50, sep = 1,  center= "dark", as_cmap=True)

In [ ]:
mfi_mtx = mfi_sortdate[['Cd41pos', 'Cd135pos', 'Cd9pos', 'Cd150pos', 'Cd36pos', 'Cd34pos', 'Cd123pos', 'Cd117pos', 'Cd48pos']].replace(np.nan, 0)

#### Hierarchical clustering

In [ ]:
sns.clustermap(mfi_mtx.transpose(), figsize=(25,10), method = 'ward', robust = True, xticklabels = True, yticklabels = True, vmin = 0, vmax = 1)

#### PCA

Use mfi_sortdate because want to keep batch info

In [ ]:
mfi_pca = mfi_sortdate.drop(['Well', 'sortdate', 'plateid'], axis = 1)
mfi_pca['pid'] = mfi['pid'].astype('category')

In [ ]:
pca = PCA(n_components=20)
# pca_result = pca.fit_transform(cluster_df.iloc[:, :-1].values)
pca_result = pca.fit_transform(mfi_pca.values)
pca_df = mfi_pca.copy()
pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
pca_df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
# fig, axes = plt.subplots(1, 2)
plt.figure(figsize=(12, 7))
sns.scatterplot(
#     ax = axes[0], 
    x="pca-one", y="pca-two",
    hue="pid",
    palette=sns.color_palette("hls", len(pca_df['pid'].unique())),
    data=pca_df,
    legend="full",
    alpha=0.7, 
    s = 100
)


# Combine Biomark and MFI data

There are 2 ways to do this:  
> Combine the data then cluster, letting flow marker contribute to hierarchy  
> Cluster the biomark data, then tack on the appropriate flow markers  

Easiest should be to combine the data then cluster, so I'll start there

## Combine data

### Merge

Use `pd.merge(how = 'left')`

In [ ]:
rna_zscoreT = rna_zscore.transpose()

In [ ]:
combined_df = rna_zscoreT.merge(mfi_mtx, how='left', right_on=mfi_mtx.index, left_index=True).drop("key_0", axis = 1).transpose()
combined_df.shape

In [ ]:
combined_df.head(3)

### Number cells per plate

In [ ]:
grouped = combined_df.transpose().copy()
pid = [str(re.match('.*?([0-9]+)$', x).group(1)) for x in grouped.index]
grouped['pid'] = pid
grouped_by_pid = grouped.groupby('pid')
for plate in grouped_by_pid.groups.keys():
    ic(plate, grouped_by_pid.get_group(plate).shape[0])

### Hierarchical cluster

#### Combined cluster:  co-sort

Cluster on biomark and mfi; mfi distributed through rows

In [ ]:
sns.clustermap(combined_df, figsize=(25,15), method = 'ward', robust = True, xticklabels = True, yticklabels = True)

#### Combined cluster: separated

Cluster on rna and mfi; plot with rna on top and mfi on bottom

In [ ]:
hm_rows = sns.clustermap(combined_df, figsize=(.5,.5), method = 'ward', robust = True, xticklabels = False, yticklabels = False, vmin = 0).dendrogram_row.reordered_ind
hm_cols = sns.clustermap(combined_df, figsize=(.5, .5), method = 'ward', robust = True, xticklabels = False, yticklabels = False, vmin = 0).dendrogram_col.reordered_ind

get the gene order from clustered rna

In [ ]:
row_order = []
for pos in hm_rows:
    newrow = combined_df.index[pos]
    row_order.append(newrow)
rna_order = [x for x in row_order if not x.endswith('pos')]
mfi_order = [x for x in row_order if x.endswith('pos')]
combined_rna = combined_df[combined_df.index.isin(rna_order)]
combined_mfi = combined_df[combined_df.index.isin(mfi_order)]

get cell order from clustered rna

In [ ]:
col_order = []
for pos in hm_cols:
    newcol = combined_df.columns[pos]
    col_order.append(newcol)

In [ ]:
sns.clustermap(combined_rna.reindex(index = rna_order, columns=col_order), figsize = (25, 15), method = 'ward', robust = True, xticklabels = False, yticklabels = True, row_cluster=False, col_cluster = False)
sns.clustermap(combined_mfi.reindex(index = mfi_order, columns=col_order), figsize = (25, 2.5), method = 'ward', robust = True, xticklabels = False, yticklabels = True, row_cluster=False, cmap = 'viridis', col_cluster = False)

## Cluster columns and rows then add mfi

### get column and row order

In [ ]:
hm_rows = sns.clustermap(rna_zscore, figsize=(1,1), method = 'ward', robust = True, xticklabels = False, yticklabels = True, vmin = 0).dendrogram_row.reordered_ind
hm_cols = sns.clustermap(rna_zscore, figsize=(25,15), method = 'ward', robust = True, xticklabels = True, yticklabels = True, vmin = 0).dendrogram_col.reordered_ind

get the gene order from clustered rna_zscore

In [ ]:
row_order = []
for pos in hm_rows:
    newrow = rna_zscore.index[pos]
    row_order.append(newrow)

get cell order from clustered rna_zscore

In [ ]:
col_order = []
for pos in hm_cols:
    newcol = rna_zscore.columns[pos]
    col_order.append(newcol)

reorder mfi columns to match rna_zscore order

In [ ]:
sns.clustermap(mfi_mtx.transpose().reindex(columns=col_order), figsize=(25,10), method = 'ward', robust = True, xticklabels = False, yticklabels = True, col_cluster=False)

### Do the whole thing in one df

In [ ]:
combined_df = rna_zscore.reindex(index=row_order, columns=col_order).append(mfi_mtx.transpose().reindex(columns=col_order), sort=False)

In [ ]:
sns.clustermap(combined_df, figsize=(25,15), method = 'ward', robust = True, xticklabels = False, yticklabels = True, row_cluster=False, col_cluster=False)

Because gene range is 0 to +infinity, and because marker range is -infinity to +infinity, graphing them on the same heatmap really throws off the colors. Will plot with rna_zscore heatmap and cell-reordered MFI heatmap (see powerpoint).

## Cluster rows-only then add MFI

Another way to do it is to cluster by genes, add the MFI, and let the columns cluster using both gene expression *and* mfi

### get column and row order

In [ ]:
hm_rows = sns.clustermap(rna_zscore, figsize=(1,1), method = 'ward', robust = True, xticklabels = False, yticklabels = True, vmin = 0).dendrogram_row.reordered_ind

get the gene order from clustered rna_zscore

In [ ]:
row_order = []
for pos in hm_rows:
    newrow = rna_zscore.index[pos]
    row_order.append(newrow)

get cell order from clustered rna_zscore

In [ ]:
col_order = []
for pos in hm_cols:
    newcol = rna_zscore.columns[pos]
    col_order.append(newcol)

### Do the whole thing in one df

combine the rna_zscore and MFI dfs, preserving rna_zscore's gene row_order. Also reindex columns in both to col_order to make sure they append properly

In [ ]:
combined_df = rna_zscore.reindex(index=row_order, columns=col_order).append(mfi_mtx.transpose().reindex(columns=col_order), sort=False)

Now plot combined df to get column order

In [ ]:
combined_cols = sns.clustermap(combined_df, figsize=(25,15), method = 'ward', robust = True, xticklabels = False, yticklabels = True, row_cluster=True, col_cluster=True).dendrogram_col.reordered_ind

Get cell order from combined clustered df (`combined_cols` is ordered by position, not cell ID)

In [ ]:
col_order = []
for pos in combined_cols:
    newcol = rna_zscore.columns[pos]
    col_order.append(newcol)

Great! Now to make the two pictures, plot rna_zscore and MFI separately, letting the rows in each cluster but preventing column clustering

reorder rna_zscore columns to match combined_df

In [ ]:
sns.clustermap(rna_zscore.reindex(columns=col_order), figsize=(25,15), method = 'ward', robust = True, xticklabels = False, yticklabels = True, col_cluster=False, row_cluster = True)

reorder mfi_mtx columns to match combined_df

In [ ]:
sns.clustermap(mfi_mtx.transpose().reindex(columns=col_order), figsize=(25,10), method = 'ward', robust = True, xticklabels = False, yticklabels = True, cmap = 'viridis', col_cluster=False, row_cluster = True)

<a id='PlottSNEData'></a>

# Plotting Data as tSNE

## tSNE of LSK, CMP, and MEP

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

### PCA

#### RNA zscore + MFI bkgrd

In [ ]:
rna_zscoreT = rna_zscore.transpose()
combined_df = rna_zscoreT.merge(mfi_mtx, how='left', right_on=mfi_mtx.index, left_index=True).drop("key_0", axis = 1).transpose()
cluster_df = combined_df.transpose().copy()

In [ ]:
pid = [str(re.match('.*?([0-9]+)$', x).group(1)) for x in cluster_df.index]
cluster_df['pid'] = pid
cluster_df.head(2)

In [ ]:
pca = PCA(n_components=20)
# pca_result = pca.fit_transform(cluster_df.iloc[:, :-1].values)
pca_result = pca.fit_transform(cluster_df.values)
pca_df = cluster_df.copy()
pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
pca_df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
# fig, axes = plt.subplots(1, 2)
plt.figure(figsize=(12, 7))
sns.scatterplot(
#     ax = axes[0], 
    x="pca-one", y="pca-two",
    hue="pid",
    palette=sns.color_palette("hls", len(pca_df['pid'].unique())),
    data=pca_df,
    legend="full",
    alpha=0.7, 
    s = 100
)


In [ ]:
ax = plt.figure(figsize=(10, 10)).gca(projection='3d')
ax.scatter(
    xs=pca_df["pca-one"], 
    ys=pca_df["pca-two"], 
    zs=pca_df["pca-three"], 
    c=pca_df["pid"].astype(int), 
    cmap= 'rainbow', 
    s = 100
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

#### RNA zscore

In [ ]:
pid = [str(re.match('.*?([0-9]+)$', x).group(1)) for x in rna_zscoreT.index]
rna_zscoreT['pid'] = pid
rna_zscoreT.head(2)

In [ ]:
pca = PCA(n_components=3)
# pca_result = pca.fit_transform(rna_df.iloc[:, :-1].values)
pca_result = pca.fit_transform(rna_zscoreT.values)
pca_df = rna_zscoreT.copy()
pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
pca_df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
# fig, axes = plt.subplots(1, 2)
plt.figure(figsize=(12, 7))
sns.scatterplot(
#     ax = axes[0], 
    x="pca-one", y="pca-two",
    hue="pid",
    palette=sns.color_palette("hls", len(pca_df['pid'].unique())),
    data=pca_df,
    legend="full",
    alpha=0.7, 
    s = 100
)


#### RNA xprsn

In [ ]:
pid = [str(re.match('.*?([0-9]+)$', x).group(1)) for x in rna_xprsn.index]
rna_xprsn['pid'] = pid
rna_xprsn.head(2)

In [ ]:
rna_xprsn.pid.unique()

In [ ]:
pca = PCA(n_components=3)
# pca_result = pca.fit_transform(rna_df.iloc[:, :-1].values)
pca_result = pca.fit_transform(rna_xprsn.values)
pca_df = rna_xprsn.copy()
pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
pca_df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
# fig, axes = plt.subplots(1, 2)
plt.figure(figsize=(12, 7))
sns.scatterplot(
#     ax = axes[0], 
    x="pca-two", y="pca-three",
    hue="pid",
    palette=sns.color_palette("hls", len(pca_df['pid'].unique())),
    data=pca_df,
    legend="full",
    alpha=0.7, 
    s = 100
)


In [ ]:
ax = plt.figure(figsize=(10, 10)).gca(projection='3d')
ax.scatter(
    xs=pca_df["pca-one"], 
    ys=pca_df["pca-two"], 
    zs=pca_df["pca-three"], 
    c=pca_df["pid"].astype(int), 
    cmap= 'Set1', 
    s = 100
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

#### PCA mfi

In [ ]:
mfi_df = combined_df.transpose().copy()
mfi_df = mfi_df.iloc[:, -9:]

In [ ]:
pid = [str(re.match('.*?([0-9]+)$', x).group(1)) for x in mfi_df.index]
mfi_df['pid'] = pid
mfi_df.head(2)

In [ ]:
pca = PCA(n_components=3)
# pca_result = pca.fit_transform(mfi_df.iloc[:, :-1].values)
pca_result = pca.fit_transform(mfi_df.values)
pca_df = mfi_df.copy()
pca_df['pca-one'] = pca_result[:,0]
pca_df['pca-two'] = pca_result[:,1] 
pca_df['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
# fig, axes = plt.subplots(1, 2)
plt.figure(figsize=(12, 7))
sns.scatterplot(
#     ax = axes[0], 
    x="pca-one", y="pca-two",
    hue="pid",
    palette=sns.color_palette("hls", len(pca_df['pid'].unique())),
    data=pca_df,
    legend="full",
    alpha=0.7, 
    s = 100
)


In [ ]:
ax = plt.figure(figsize=(10, 10)).gca(projection='3d')
ax.scatter(
    xs=pca_df["pca-one"], 
    ys=pca_df["pca-two"], 
    zs=pca_df["pca-three"], 
    c=pca_df["pid"].astype(int), 
    cmap= 'Set1', 
    s = 100
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

### TSNE

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
# tsne_results = tsne.fit_transform(cluster_df.iloc[:,:-1])
tsne_results = tsne.fit_transform(cluster_df)

In [ ]:
tsne_df = cluster_df.copy()
tsne_df['tsne-2d-one'] = tsne_results[:,0]
tsne_df['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="pid",
    palette=sns.color_palette("hls", len(tsne_df.pid.unique())),
    data=tsne_df,
    legend="full",
    alpha=1,
    s = 100
)

In [ ]:
X = mnist.data / 255.0
y = mnist.target
print(X.shape, y.shape)

In [ ]:
feat_cols = [ 'pixel'+str(i) for i in range(X.shape[1]) ]
df = pd.DataFrame(X,columns=feat_cols)
df['y'] = y
df['label'] = df['y'].apply(lambda i: str(i))
X, y = None, None
print('Size of the dataframe: {}'.format(df.shape))

In [ ]:
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])

In [ ]:
plt.gray()
fig = plt.figure( figsize=(16,7) )
for i in range(0,15):
    ax = fig.add_subplot(3,5,i+1, title="Digit: {}".format(str(df.loc[rndperm[i],'label'])) )
    ax.matshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float))
plt.show()

In [ ]:
df

In [ ]:
df

In [ ]:
rndperm

# Normalization Notes

[Bioconductor Support Article](https://support.bioconductor.org/p/34182/)

Other approaches

[Published data analysis for 96.96 Dynamic Array IFC](https://www.gene-quantification.de/livak-methods-59-transcriptional-biomarkers-2013.pdf)  
* Data analyzed using Fluidigm Real-Time PCR Analysis software  
    * Used linear (derivative) baseline correction method  
    * Used Auto (global) Ct threshold method (alt. 0.01)
    * Ct range 12 - 28 cycles
    * Export Cq values  

[G. Guo, Dev. Cell; 18 (2010) 675–685.](https://www.sciencedirect.com/science/article/pii/S1534580710001103#sec4)
* Relative expression determined by subtracting Ct value from "assumed baseline of 28"
* Remove cells with absent/low endogenous controls (~10%)
* Normalize these by subtracting average of ActB+GAPDH expression levels

[geNorm method](https://doi.org/10.1186/gb-2002-3-7-research0034)
* Average control genes using __geometric mean__ (not arithmatic mean)
    * genorm support stops with python 2.7 
    * R may have an implementation for it via [NormqPCR](https://www.bioconductor.org/packages/release/bioc/html/NormqPCR.html)
    
[ERgene](https://www.nature.com/articles/s41598-020-75586-5)
* Python library to for gene nomralization [github](https://github.com/Starlitnightly/ERgene)

[Beth Psaila's Approach](https://doi.org/10.1186/s13059-016-0939-7)
* Exclude assays with:
    * LOD (limit of detection) >= 40
* Exclude cells with:
    * >70 failed assays
    * B2M >= 13
    * GAPDH >= 15
    * Cells with mean Ct > 20
* Norm to average of [B2M & GAPDH]
* Expression = 2^-(NormCt)
* Exclude HKgenes

Going with Beth's method because it's the most thoroughly described.
Might also gofor quantile nomrlization